In [129]:
from nltk.corpus import brown
from collections import OrderedDict
import operator

In [130]:
##################### Declarations ##############
data = []
raw_data = brown.tagged_words()

Vocab = set()
word_tag_pair = {}
transition = {}
emission = {}
emission_prob = {}
emit_word = {}

transistion_trigram = {}
transistion_bigram = {}
transistion_prob = {}
context = {}
emit = {}

tag_list = []

SOS = "<s>"
SSOS = "<ss>"
EOS = "</s>"

kk = 0.1
l1 = 0.1
l2 = 0

In [131]:
##################### Pre processing ##############
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def pre_process(data):
    #print(type(data))
    data1 = []
    sample0 = (SSOS,"$SStart")
    sample = (SOS,"$Start")
    sample1 = (EOS,"$END")
    
    data1.append(sample0)
    data1.append(sample)
    
    for i in range(len(data)):  
        if hasNumbers(data[i][0]):
            continue
        
        elif data[i][0] == '.':
            data1.append(tuple(sample1))
            data1.append(tuple(sample0))
            data1.append(tuple(sample))
            
        elif len(data[i][1]) == 1:
            continue
        
        elif data[i][0]==data[i][1]:
            continue
        
        else:
            if data[i][1].find('-')!=-1:
                temp = data[i][1].split("-")
                temp[0] = temp[0].replace("*","")
                temp[0] = temp[0].replace("$","")
                temp[0] = temp[0].replace(")","")
                temp[0] = temp[0].replace("(","")
                temp[0] = temp[0].replace(":","")
                temp[0] = temp[0].replace(",","")
                temp[0] = temp[0].replace(".","")
                
                if len(temp[0]) > 1:
                    data1.append((data[i][0],temp[0][:2]))
            
            elif data[i][1].find('+')!=-1:
                temp = data[i][1].split("+")
                temp[0] = temp[0].replace("*","")
                temp[0] = temp[0].replace("$","")
                temp[0] = temp[0].replace(")","")
                temp[0] = temp[0].replace("(","")
                temp[0] = temp[0].replace(":","")
                temp[0] = temp[0].replace(".","")
                temp[0] = temp[0].replace(",","")
                
                if len(temp[0]) > 1:
                    data1.append((data[i][0],temp[0][:2]))   
            
            else:
                temp = data[i][1].replace("*","")
                temp = temp.replace("*","")
                temp = temp.replace("$","")
                temp = temp.replace(")","")
                temp = temp.replace("(","")
                temp = temp.replace(":","")
                temp = temp.replace(".","")
                temp = temp.replace(",","")
                
                if len(temp) > 1:
                    data1.append((data[i][0],temp[:2]))
                
    data1.pop()
    data1.pop()
    processed_data = []
    temp = []
    
    for i in range(len(data1)):
        #print(data1[i][1])
        if data1[i][0] == EOS:
            temp.append(tuple(data1[i]))
            processed_data.append(temp)
            temp = []
        else:
            temp.append(data1[i])
    
    return processed_data

In [132]:
##################### Populate dictionaries ##############
from collections import OrderedDict
def build_dicts(sentences):
    global Vocab, transistion_bigram, transistion_trigram, context, emit, word_tag_pair
    global tag_list
    
    for sentence in sentences:
        prev1_word = None
        prev2_word = None
    
        transistion_bigram[(sentence[0][1],sentence[1][1])] = transistion_bigram.get((sentence[0][1],sentence[1][1]),0) + 1
        
        for words in sentence:
            Vocab.add(words[0])
            
            context[words[1]] = context.get(words[1],0) + 1
            emit[(words[1],words[0])] = emit.get((words[1],words[0]),0) + 1
            emission[(words[1],words[0])] = emission.get((words[1],words[0]),0) + 1
            word_tag_pair[(words[1],words[0])] = word_tag_pair.get((words[1],words[0]),0) + 1
            
            if prev1_word != None and prev2_word != None:
                transistion_trigram[(prev1_word,prev2_word,words[1])] = transistion_trigram.get((prev1_word,prev2_word,words[1]),0) + 1
                transistion_bigram[(prev2_word,words[1])] = transistion_bigram.get((prev2_word,words[1]),0) + 1
                
            
            prev1_word = prev2_word
            prev2_word = words[1] 
        
    for keys,values in context.items():
        tag_list.append(keys)
        

#     for keys,values in context.items():
#         print(str(keys) + "  " + str(values))
#     print("\n\n----------------------------------------------------\n\n")    
#     for keys,values in emission.items():
#         print(str(keys) + "  " + str(values))
#     print("\n\n----------------------------------------------------\n\n")            
    
#     for keys,values in transistion_trigram.items():
#         print(str(keys) + "  " + str(values))    
#     print("\n\n----------------------------------------------------\n\n")        
    

In [133]:
############################# Generate Probability ###########################
def gen_prob():
    global transistion_bigram, transistion_trigram, emission_prob, transition_prob
    for keys,values in transistion_trigram.items():
        deno = transistion_bigram.get((keys[0],keys[1]),0)
        transistion_prob[keys] = values/float(deno)
    
    for keys,values in emission.items():    
        deno = transistion_bigram.get((keys[0],keys[1]),0)
        emission_prob[keys] = values/float(deno)

In [134]:
############################# VITERBI ALGO ###########################
def calc_trans_prob(trigram):
    
    global emission_prob, transition_trigram, transistion_bigram
    global kk,l1,l2
    
    wn_0 = trigram[0]
    wn_1 = trigram[1]
    wn_2 = trigram[2]

    x1 = l1*1.0*(transistion_trigram.get( (wn_0, wn_1, wn_2) , 0) + kk)/(transistion_bigram.get( (wn_0, wn_1) , 0) + kk*len(context.keys()) )
    x2 = l2*1.0*(transistion_bigram.get( (wn_1, wn_2) , 0) + kk)/(context.get(wn_1, 0) + kk*len(context.keys()) )
    return x1+x2
    
def calc_emi_prob(trigram):  
    global emission_prob, transition_prob, transistion_bigram
    global kk,l1,l2
    
    wn_0 = trigram[0]
    wn_1 = trigram[1]

    x1 = l1*1.0*(emission.get( (wn_0, wn_1) , 0) + kk)/(context.get(wn_0, 0) + kk*len(Vocab) )
    return x1

def viterbi(sentence):
    global tag_list
    dp = []
    tag_seq = []

    #INITIALIZE DATA STRUCT
    for i in range(len(sentence)):
        dp.append([])
        tag_seq.append([])
        for j in range(len(tag_list)):
            dp[i].append([])
            tag_seq[i].append([])
            for each in tag_list:
                dp[i][j].append(0)
                tag_seq[i][j].append(0)

    #INITIALIZE DP
    for i in range(len(tag_list)):
        tag_gram = ("$SStart", "$Start", tag_list[i])
        trans = calc_trans_prob(tag_gram)
        word_gram = ("$SStart", SOS)
        emit = calc_emi_prob(word_gram)
        dp[0][1][i] = trans*emit

    #DP
    for p in range(1, len(sentence)):
        #print(sentence[p])
        for k in range(len(tag_list)):
            for j in range(len(tag_list)):
                maximum = -1
                word_gram = (tag_list[k], sentence[p][0])
                emit = calc_emi_prob(word_gram)
                
                for i in range(len(tag_list)):
                    tag_gram = (tag_list[i], tag_list[j], tag_list[k])
                    trans = calc_trans_prob(tag_gram)
                    temp = dp[p-1][i][j] * trans
                
                    if temp >= maximum:
                        maximum = temp
                        max_pos = i
                
                dp[p][j][k] = maximum * emit
                tag_seq[p][j][k] = max_pos

    #RECONSTRUCT TAG SEQUENCE
    final_tags = []
    maximum = -1
    for i in range(len(tag_list)):
        for j in range(len(tag_list)):
            temp = dp[-1][i][j]
            
            if temp > maximum:
                maximum = temp
                max_i = i
                max_j = j
    
    final_tags.append(tag_list[max_j])
    final_tags.append(tag_list[max_i])
    p = len(sentence) - 1
    
    while p >= 0:
        next = tag_seq[p][max_i][max_j]
        max_j = max_i
        max_i = next
        final_tags.append(tag_list[next])
        p -= 1
    
    final_tags.reverse()
    return final_tags


In [135]:
############################# MAIN ###########################
for i in raw_data:
    data.append(i)
processed_data = pre_process(data)

train_data = []
test_data = []

for i in range(len(processed_data)):
    if i <= 0.8*len(processed_data):
        train_data.append(processed_data[i])
    else:
        test_data.append(processed_data[i])
        

build_dicts(train_data)
print("Vocab size",len(Vocab))
print("Number of Tags: ",len(tag_list))
print("\nTagList\n",tag_list)

Vocab size 49138
Number of Tags:  36

TagList
 ['NN', 'NI', 'RB', 'WP', 'RP', 'DT', 'UH', 'NR', 'FW', 'RN', 'CS', '$SStart', 'PP', 'OD', 'AP', 'AB', 'BE', 'HV', 'TO', 'JJ', 'AT', 'WR', 'NP', 'DO', '$END', 'VB', 'IN', 'MD', 'QL', '$Start', 'CD', 'EX', 'CC', 'WQ', 'PN', 'WD']


In [136]:
#################### Top 50 ########################
tag_top50 = {}

for key,val in sorted(word_tag_pair.items(),key=operator.itemgetter(1),reverse=True):
    temp = tag_top50.get(key[0],{})
    if len(temp)<51:
        temp[key[1]] = val
        tag_top50[key[0]] = temp
        
for k,v in tag_top50.items():
    print("\n--------------------------\nTAG: ",k)
    for k1,v1 in sorted(v.items(),key=operator.itemgetter(1),reverse=True):
        print(k1,"\t",v1)


--------------------------
TAG:  NR
today 	 228
home 	 212
South 	 105
Sunday 	 93
West 	 85
yesterday 	 75
Monday 	 63
Saturday 	 59
Friday 	 56
left 	 56
Tuesday 	 55
right 	 51
north 	 50
south 	 44
North 	 43
tomorrow 	 42
East 	 41
west 	 38
downtown 	 36
Today 	 35
east 	 34
today's 	 34
Wednesday 	 31
Thursday 	 31
tonight 	 25
southeast 	 9
Sundays 	 8
Today's 	 7
Southwest 	 7
northwest 	 7
Northeast 	 6
Sunday's 	 6
Home 	 5
yesterday's 	 5
to-day 	 4
southwest 	 4
Tonight 	 4
South's 	 4
Saturday's 	 3
northeast 	 3
West's 	 3
tomorrow's 	 3
Fridays 	 3
Monday's 	 3
Northwest 	 2
Yesterday 	 2
tonight's 	 2
Downtown 	 2
Friday's 	 2
Tomorrow 	 2
Saturdays 	 1

--------------------------
TAG:  NN
time 	 1249
Af 	 993
man 	 839
years 	 836
people 	 693
way 	 684
world 	 626
year 	 612
men 	 611
life 	 576
work 	 523
state 	 510
day 	 488
number 	 454
States 	 439
fact 	 418
part 	 406
course 	 386
system 	 386
school 	 378
place 	 373
program 	 365
group 	 351
use 	 340
busin

doesn't 	 59
Does 	 36
Did 	 22
Didn't 	 5
done 	 4
doo 	 2
Doesn't 	 2
dost 	 1

--------------------------
TAG:  $END
</s> 	 39477

--------------------------
TAG:  VB
said 	 1174
made 	 934
make 	 633
used 	 566
see 	 536
get 	 486
know 	 468
United 	 457
found 	 444
take 	 443
came 	 442
come 	 438
go 	 433
say 	 353
given 	 338
become 	 332
put 	 329
went 	 326
find 	 326
called 	 318
give 	 311
think 	 299
took 	 288
told 	 287
asked 	 273
set 	 270
got 	 269
left 	 251
going 	 250
done 	 250
taken 	 248
knew 	 242
began 	 235
seems 	 234
felt 	 231
thought 	 230
want 	 219
gave 	 216
seemed 	 216
saw 	 215
became 	 212
seen 	 212
held 	 212
provide 	 209
known 	 207
looked 	 203
let 	 201
turned 	 198
look 	 196
seem 	 196
keep 	 195

--------------------------
TAG:  IN
of 	 32616
in 	 16937
to 	 9527
for 	 7492
with 	 5912
on 	 4917
by 	 4686
at 	 4063
from 	 3613
In 	 1653
into 	 1398
about 	 937
through 	 715
over 	 666
between 	 658
under 	 564
against 	 493
For 	 471
than 	

In [137]:
#####################################  VITERBI #######################

for it1 in range(1,10):
    for it2 in range(1,10):
        for it3 in range(0,10):

            if (it2 + it3) > 10:
                break

            sent_accuracy = 0
            correct_tokens = 0
            total_tokens = 0
            avg = 0
            
            # Finding accuracy on test data
            for sent in test_data:
                result = viterbi(sent)[2:]
                answer = [word[1] for word in sent]
            
                if result == answer:
                    sent_accuracy += 1
                else:
                    pass
                    print("\nPredicted", result)
                    print ("Actual", answer)
                
                total_tokens += len(result)
                counter = 0
                for i in range(len(result)):
                    if result[i] == answer[i]:
                        counter += 1
                print("Accuracy: ",100*counter/len(sent))


Predicted ['PP', '$Start', 'EX', 'BE', 'AP', 'NN', 'IN', 'AT', 'NN', 'CC', 'AT', 'NN', 'BE', 'VB', '$END']
Actual ['$SStart', '$Start', 'EX', 'BE', 'JJ', 'NN', 'IN', 'AT', 'NN', 'CC', 'AT', 'NN', 'BE', 'VB', '$END']
Accuracy:  86.66666666666667

Predicted ['PP', '$Start', 'IN', 'JJ', 'NN', 'PP', 'MD', 'VB', 'IN', 'AT', 'JJ', 'NN', 'IN', 'JJ', 'NP', 'CC', 'NP', 'NP', 'IN', 'NN', 'CC', 'JJ', 'NN', '$END']
Actual ['$SStart', '$Start', 'IN', 'JJ', 'NN', 'PP', 'MD', 'VB', 'IN', 'AT', 'JJ', 'NN', 'IN', 'JJ', 'NP', 'CC', 'NP', 'NP', 'IN', 'NN', 'CC', 'JJ', 'NN', '$END']
Accuracy:  95.83333333333333

Predicted ['PP', '$Start', 'CC', 'PP', 'VB', 'PP', 'BE', 'JJ', 'WR', 'AP', 'JJ', 'NN', 'VB', 'IN', 'JJ', 'NN', 'CC', 'PP', 'MD', 'VB', 'PP', 'CS', 'PP', 'VB', 'PP', 'RB', 'IN', 'AT', 'JJ', 'NN', '$END']
Actual ['$SStart', '$Start', 'CC', 'PP', 'VB', 'PP', 'BE', 'JJ', 'WR', 'RB', 'JJ', 'NN', 'VB', 'IN', 'JJ', 'NN', 'CC', 'PP', 'MD', 'VB', 'PP', 'CS', 'PP', 'VB', 'PP', 'RB', 'IN', 'AT', 'JJ', 'NN',


Predicted ['PP', '$Start', 'CS', 'CS', 'PP', 'BE', 'RB', 'VB', 'TO', 'VB', 'CC', 'RB', 'VB', 'RB', '$END']
Actual ['$SStart', '$Start', 'IN', 'CS', 'PP', 'BE', 'RB', 'VB', 'TO', 'VB', 'CC', 'RB', 'VB', 'RB', '$END']
Accuracy:  86.66666666666667

Predicted ['PP', '$Start', 'NR', 'EX', 'BE', 'AT', 'AP', 'VB', 'NN', 'IN', 'AB', 'JJ', 'NP', 'CS', 'PP', 'VB', 'IN', '$END']
Actual ['$SStart', '$Start', 'NR', 'NP', 'BE', 'AT', 'AP', 'VB', 'NN', 'IN', 'AB', 'JJ', 'NP', 'CS', 'PP', 'VB', 'IN', '$END']
Accuracy:  88.88888888888889

Predicted ['PP', '$Start', 'RB', 'PP', 'VB', 'AT', 'JJ', 'NN', 'IN', 'NN', 'CS', 'NP', 'VB', 'IN', 'CC', 'RB', 'PP', 'AT', 'QL', 'VB', 'CC', 'VB', 'NN', 'IN', 'AT', 'NN', '$END']
Actual ['$SStart', '$Start', 'RB', 'PP', 'VB', 'AT', 'JJ', 'NN', 'IN', 'NN', 'CS', 'NP', 'RB', 'RB', 'CC', 'RB', 'PP', 'AT', 'QL', 'VB', 'CC', 'VB', 'NN', 'IN', 'AT', 'NN', '$END']
Accuracy:  88.88888888888889

Predicted ['PP', '$Start', 'DT', 'WP', 'VB', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NN', 


Predicted ['PP', '$Start', 'AT', 'NN', 'VB', 'IN', 'AT', 'JJ', 'NN', 'CC', 'PP', 'MD', 'VB', 'PP', 'IN', 'PP', '$END']
Actual ['$SStart', '$Start', 'AT', 'NN', 'VB', 'IN', 'AT', 'JJ', 'NN', 'CC', 'PP', 'MD', 'VB', 'PP', 'IN', 'PP', '$END']
Accuracy:  94.11764705882354

Predicted ['PP', '$Start', 'UH', 'RB', 'PP', 'QL', 'JJ', 'PP', 'JJ', 'CS', 'IN', 'AT', 'FW', 'FW', '$END']
Actual ['$SStart', '$Start', 'UH', 'RB', 'PP', 'QL', 'JJ', 'PP', 'JJ', 'CS', 'IN', 'AT', 'NN', 'NN', '$END']
Accuracy:  80.0

Predicted ['PP', '$Start', 'RN', 'WP', 'WR', 'DT', 'WD', 'VB', 'PP', 'AT', 'NN', 'VB', 'PP', 'IN', 'AT', 'NN', 'AT', 'QL', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', '$END']
Actual ['$SStart', '$Start', 'WD', 'DT', 'WR', 'DT', 'WD', 'VB', 'PP', 'AT', 'NN', 'VB', 'PP', 'RB', 'AT', 'NN', 'AT', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', '$END']
Accuracy:  76.0

Predicted ['PP', '$Start', 'PP', 'VB', 'RB', 'BE', 'NN', 'IN', 'WD', 'PP', 'BE', 'RB', 'JJ', '$END']
Actual ['$SStart', '$Start', 'PP', 'VB', '

KeyboardInterrupt: 